## Imports


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import Counter
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
from loguru import logger as lg
from rich import print as rprint
from rich.console import Console
from rich import get_console

# Rich Jupyter fix
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.sheet_aruco import SheetAruco
from snap_fit.puzzle.sheet_manager import SheetManager

## Helper: Load Puzzle Data


In [ ]:
def load_puzzle_sheets(puzzle_name: str) -> SheetManager:
    """Load puzzle sheets for a given puzzle name (e.g., 'sample_puzzle_v1')."""
    # Configure ArUco detection
    board_config = ArucoBoardConfig(
        markers_x=7,
        markers_y=5,
        marker_length=100,
        marker_separation=100,
    )
    detector_config = ArucoDetectorConfig(board=board_config)

    # Initialize sheet loader
    sheet_aruco = SheetAruco(detector_config)
    aruco_loader = partial(sheet_aruco.load_sheet, min_area=5_000)

    # Load puzzle sheets
    paths = get_snap_fit_paths()
    data_dir = paths.data_fol / puzzle_name / "sheets"
    lg.info(f"Loading data from {data_dir}")

    manager = SheetManager()
    manager.add_sheets(
        folder_path=data_dir,
        pattern="*.png",
        loader_func=aruco_loader,
    )

    lg.info(
        f"Loaded {len(manager.get_sheets_ls())} sheets with "
        f"{len(manager.get_pieces_ls())} pieces"
    )
    return manager

## Load v1 and v2 Puzzle Data


In [ ]:
manager_v1 = load_puzzle_sheets("sample_puzzle_v1")

In [ ]:
manager_v2 = load_puzzle_sheets("sample_puzzle_v2")

In [ ]:
# Compute expected segment counts for a 6x8 puzzle
ROWS, COLS = 6, 8

total_pieces = ROWS * COLS  # 48
total_segments = total_pieces * 4  # 192

# EDGE segments are on the puzzle boundary (perimeter)
perimeter_segments = 2 * (ROWS + COLS)  # 28

# Internal segments should be IN or OUT (each internal edge has one of each)
internal_horizontal_edges = ROWS * (COLS - 1)  # 42
internal_vertical_edges = (ROWS - 1) * COLS  # 40
total_internal_edges = internal_horizontal_edges + internal_vertical_edges  # 82

# Each internal edge contributes one IN and one OUT segment
expected_in = total_internal_edges  # 82
expected_out = total_internal_edges  # 82
expected_edge = perimeter_segments  # 28
expected_weird = 0  # ideally none!

rprint("[bold]Expected Segment Distribution for 6x8 Puzzle[/bold]")
rprint(f"  Total segments: {total_segments}")
rprint(
    f"  EDGE (boundary): {expected_edge} ({100 * expected_edge / total_segments:.1f}%)"
)
rprint(f"  IN:   {expected_in} ({100 * expected_in / total_segments:.1f}%)")
rprint(f"  OUT:  {expected_out} ({100 * expected_out / total_segments:.1f}%)")
rprint(f"  WEIRD (target): {expected_weird} (0.0%)")

## Analyze Shape Distribution


In [ ]:
def analyze_shapes(manager: SheetManager, label: str) -> Counter:
    """Count segment shapes across all pieces."""
    shapes = []
    for piece in manager.get_pieces_ls():
        for seg in piece.segments.values():
            shapes.append(seg.shape)

    counts = Counter(shapes)
    total = sum(counts.values())

    rprint(f"\n[bold]{label}[/bold]")
    rprint(f"  Total segments: {total}")
    for shape, count in counts.most_common():
        pct = 100 * count / total
        rprint(f"  {shape.name:>6}: {count:>3} ({pct:5.1f}%)")

    return counts

In [ ]:
counts_v1 = analyze_shapes(manager_v1, "sample_puzzle_v1")
counts_v2 = analyze_shapes(manager_v2, "sample_puzzle_v2")

## Visualize WEIRD Segments

Let's look at segments classified as WEIRD to understand the failure mode.


In [ ]:
from snap_fit.image.segment import SegmentShape


def get_weird_segments(manager: SheetManager, max_count: int = 10):
    """Get a sample of WEIRD segments for visualization."""
    weird_segments = []
    for piece in manager.get_pieces_ls():
        for seg_id, seg in piece.segments.items():
            if seg.shape == SegmentShape.WEIRD:
                weird_segments.append((piece.piece_id, seg_id, seg))
                if len(weird_segments) >= max_count:
                    return weird_segments
    return weird_segments


weird_v1 = get_weird_segments(manager_v1, max_count=6)
rprint(f"Found {len(weird_v1)} WEIRD segments to analyze")

In [ ]:
def plot_segment_points(seg, ax, title: str):
    """Plot segment points showing the classification problem."""
    # seg.points has shape (N, 1, 2) - OpenCV contour format
    pts = seg.points.squeeze()  # Now (N, 2)

    # Plot raw points
    ax.plot(pts[:, 0], pts[:, 1], "b-", linewidth=2, label="Segment")
    ax.scatter(pts[0, 0], pts[0, 1], c="green", s=100, zorder=5, label="Start")
    ax.scatter(pts[-1, 0], pts[-1, 1], c="red", s=100, zorder=5, label="End")

    ax.set_title(f"{title}\nShape: {seg.shape.name}")
    ax.set_aspect("equal")
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

In [ ]:
# Visualize WEIRD segments
if weird_v1:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()

    for i, (piece_id, seg_id, seg) in enumerate(weird_v1[:6]):
        plot_segment_points(seg, axes[i], f"Piece {piece_id}, Seg {seg_id}")

    plt.tight_layout()
    plt.show()

## Analyze Point Distribution (Understanding the Classification)

The current algorithm counts points beyond `flat_th=20` on each side of the center line.
Let's see the actual distributions.


In [ ]:
def analyze_segment_distribution(seg):
    """Analyze segment point distribution relative to center line.

    Returns dict with statistics about point distribution.
    """
    # seg.points has shape (N, 1, 2) - OpenCV contour format
    pts = seg.points.squeeze()  # Now (N, 2)

    # Transform to align start-end with x-axis (mimicking _compute_shape)
    start, end = pts[0], pts[-1]
    direction = end - start
    length = np.linalg.norm(direction)

    if length < 1e-6:
        return None

    # Rotation to align with x-axis
    angle = np.arctan2(direction[1], direction[0])
    cos_a, sin_a = np.cos(-angle), np.sin(-angle)

    # Transform points
    centered = pts - start
    rotated = np.column_stack(
        [
            centered[:, 0] * cos_a - centered[:, 1] * sin_a,
            centered[:, 0] * sin_a + centered[:, 1] * cos_a,
        ]
    )

    # Y values are perpendicular distances from center line
    y_vals = rotated[:, 1]

    # Current thresholds
    flat_th = 20
    count_th = 5

    out_count = (y_vals < -flat_th).sum()  # Points below line
    in_count = (y_vals > flat_th).sum()  # Points above line

    return {
        "shape": seg.shape,
        "y_vals": y_vals,
        "mean_y": np.mean(y_vals),
        "std_y": np.std(y_vals),
        "min_y": np.min(y_vals),
        "max_y": np.max(y_vals),
        "out_count": out_count,
        "in_count": in_count,
        "is_weird": out_count > count_th and in_count > count_th,
        "signed_area": np.trapz(y_vals),
        "length": length,
    }

In [ ]:
# Analyze all segments
all_stats_v1 = []
for piece in manager_v1.get_pieces_ls():
    for seg_id, seg in piece.segments.items():
        stats = analyze_segment_distribution(seg)
        if stats:
            stats["piece_id"] = piece.piece_id
            stats["seg_id"] = seg_id
            all_stats_v1.append(stats)

rprint(f"Analyzed {len(all_stats_v1)} segments")

In [ ]:
# Compare distributions by shape type
from snap_fit.image.segment import SegmentShape

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

shape_types = [SegmentShape.IN, SegmentShape.OUT, SegmentShape.EDGE, SegmentShape.WEIRD]
colors = ["blue", "green", "orange", "red"]

for ax, shape, color in zip(axes.flatten(), shape_types, colors):
    shape_stats = [s for s in all_stats_v1 if s["shape"] == shape]

    if shape_stats:
        mean_ys = [s["mean_y"] for s in shape_stats]
        signed_areas = [s["signed_area"] for s in shape_stats]

        ax.scatter(mean_ys, signed_areas, c=color, alpha=0.6, s=50)
        ax.axhline(y=0, color="gray", linestyle="--", alpha=0.5)
        ax.axvline(x=0, color="gray", linestyle="--", alpha=0.5)

    ax.set_title(f"{shape.name} (n={len(shape_stats)})")
    ax.set_xlabel("Mean Y (perpendicular displacement)")
    ax.set_ylabel("Signed Area")
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Compare Classification Approaches

Let's test alternative classification methods on the existing data.


In [ ]:
def classify_by_mean(stats, threshold=10):
    """Option B: Classify by mean displacement."""
    mean_y = stats["mean_y"]
    if mean_y < -threshold:
        return "OUT"
    elif mean_y > threshold:
        return "IN"
    else:
        return "EDGE"


def classify_by_area(stats, threshold=500):
    """Option B: Classify by signed area."""
    area = stats["signed_area"]
    if area < -threshold:
        return "OUT"
    elif area > threshold:
        return "IN"
    else:
        return "EDGE"


def classify_adaptive(stats):
    """Option A: Adaptive thresholds based on segment stats."""
    y_vals = stats["y_vals"]
    flat_th = max(10, np.std(y_vals) * 1.5)
    count_th = max(3, len(y_vals) * 0.05)

    out_count = (y_vals < -flat_th).sum()
    in_count = (y_vals > flat_th).sum()

    # Convert numpy bools to Python bools for pattern matching
    is_out = bool(out_count > count_th)
    is_in = bool(in_count > count_th)

    match (is_out, is_in):
        case (True, False):
            return "OUT"
        case (False, True):
            return "IN"
        case (False, False):
            return "EDGE"
        case (True, True):
            return "WEIRD"

In [ ]:
# Compare classification methods
methods = {
    "Current": lambda s: s["shape"].name,
    "Mean (th=10)": lambda s: classify_by_mean(s, threshold=10),
    "Mean (th=15)": lambda s: classify_by_mean(s, threshold=15),
    "Area (th=50)": lambda s: classify_by_area(s, threshold=50),
    "Area (th=500)": lambda s: classify_by_area(s, threshold=500),
    "Area (th=1000)": lambda s: classify_by_area(s, threshold=1000),
    "Adaptive": classify_adaptive,
}

rprint("\n[bold]Classification Method Comparison (v1)[/bold]")
if not all_stats_v1:
    rprint("[red]No stats available - rerun previous cells[/red]")
else:
    for method_name, classify_fn in methods.items():
        classifications = [classify_fn(s) for s in all_stats_v1]
        counts = Counter(classifications)
        total = len(classifications)
        weird_pct = 100 * counts.get("WEIRD", 0) / total if total > 0 else 0
        rprint(
            f"  {method_name:15s}: WEIRD={counts.get('WEIRD', 0):>3} ({weird_pct:5.1f}%), "
            f"IN={counts.get('IN', 0):>3}, OUT={counts.get('OUT', 0):>3}, EDGE={counts.get('EDGE', 0):>3}"
        )

## ⚠️ Critical Insight: WEIRD is Safer Than Misclassification

**Misclassifying a segment (e.g., IN→OUT) is WORSE than classifying it as WEIRD.**

- **Wrong IN/OUT**: Matcher computes similarity with wrong polarity → bad matches
- **WEIRD fallback**: Triggers flexible matching that doesn't assume shape

**Algorithm priority:**

1. Minimize false IN/OUT classifications
2. Accept some WEIRD as safe fallback
3. Only classify IN/OUT when confident


In [ ]:
# Analyze misclassification risk: check if mean_y sign matches shape classification
# A misclassification is when mean_y suggests one direction but classification says opposite


def check_misclassification(stats, method_fn):
    """Check if classification contradicts the dominant direction."""
    mean_y = stats["mean_y"]
    classification = method_fn(stats)

    # Determine "ground truth" direction from mean_y
    # Positive mean_y → segment bulges IN, Negative → bulges OUT
    CONFIDENCE_TH = 5  # Only consider clear cases

    if abs(mean_y) < CONFIDENCE_TH:
        # Ambiguous - can't determine ground truth
        return "ambiguous"

    expected = "IN" if mean_y > 0 else "OUT"

    if classification == "WEIRD":
        return "weird_safe"  # WEIRD is always safe
    elif classification == "EDGE":
        if abs(mean_y) > 10:  # Should not be EDGE if clearly displaced
            return "FALSE_EDGE"  # Classifying IN/OUT as EDGE is BAD
        return "edge_ok"
    elif classification == expected:
        return "correct"
    else:
        return "WRONG_POLARITY"  # Wrong direction - BAD!


rprint("\n[bold red]Misclassification Analysis[/bold red]")
rprint("Errors: WRONG_POLARITY = IN↔OUT swap, FALSE_EDGE = IN/OUT classified as EDGE")
rprint(f"(Expected max EDGE = {expected_edge}, any excess is likely FALSE_EDGE)\n")

for method_name, classify_fn in methods.items():
    results = [check_misclassification(s, classify_fn) for s in all_stats_v1]
    counts = Counter(results)
    total = len(results)

    # Count classifications
    classifications = [classify_fn(s) for s in all_stats_v1]
    class_counts = Counter(classifications)
    edge_count = class_counts.get("EDGE", 0)
    excess_edge = max(0, edge_count - expected_edge)

    wrong_polarity = counts.get("WRONG_POLARITY", 0)
    false_edge = counts.get("FALSE_EDGE", 0)
    total_errors = wrong_polarity + false_edge
    weird_safe = counts.get("weird_safe", 0)
    correct = counts.get("correct", 0)

    # Status based on total errors
    if total_errors == 0:
        status = "🟢 SAFE"
    elif total_errors <= 5:
        status = "🟡 OK"
    else:
        status = "🔴 BAD"

    rprint(
        f"  {method_name:15s}: wrong_pol={wrong_polarity:>3} false_edge={false_edge:>3} | "
        f"EDGE={edge_count:>3} (excess={excess_edge:>2}) | weird={class_counts.get('WEIRD', 0):>3} | {status}"
    )

## Visualize Classification Boundaries

Let's see how each method draws decision boundaries in the mean_y vs signed_area space.


In [ ]:
# Visualize how different methods classify segments
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

selected_methods = [
    "Current",
    "Mean (th=10)",
    "Mean (th=15)",
    "Area (th=500)",
    "Area (th=1000)",
    "Adaptive",
]
class_colors = {"IN": "blue", "OUT": "green", "EDGE": "orange", "WEIRD": "red"}

for ax, method_name in zip(axes, selected_methods):
    classify_fn = methods[method_name]

    for stats in all_stats_v1:
        cls = classify_fn(stats)
        color = class_colors.get(cls, "gray")
        ax.scatter(stats["mean_y"], stats["signed_area"], c=color, alpha=0.5, s=30)

    # Add decision boundary lines for reference
    ax.axhline(y=0, color="gray", linestyle="--", alpha=0.3)
    ax.axvline(x=0, color="gray", linestyle="--", alpha=0.3)

    # Count by class
    classifications = [classify_fn(s) for s in all_stats_v1]
    counts = Counter(classifications)

    ax.set_title(
        f"{method_name}\nIN={counts.get('IN', 0)} OUT={counts.get('OUT', 0)} "
        f"EDGE={counts.get('EDGE', 0)} WEIRD={counts.get('WEIRD', 0)}"
    )
    ax.set_xlabel("Mean Y")
    ax.set_ylabel("Signed Area")
    ax.grid(True, alpha=0.3)

# Add legend
from matplotlib.patches import Patch

legend_elements = [Patch(facecolor=c, label=l) for l, c in class_colors.items()]
fig.legend(handles=legend_elements, loc="upper right", bbox_to_anchor=(0.99, 0.99))

plt.tight_layout()
plt.show()

## Proposed Safe Classification: Area with WEIRD Fallback

A conservative approach that:

1. Uses signed area for clear IN/OUT
2. Falls back to WEIRD when evidence is weak or contradictory


In [ ]:
def classify_safe_area(stats, area_th=500, edge_th=100):
    """Safe classification: only classify IN/OUT when confident, else WEIRD.

    Args:
        area_th: Threshold for confident IN/OUT classification
        edge_th: Below this, classify as EDGE (flat segment)
    """
    area = stats["signed_area"]
    mean_y = stats["mean_y"]

    # Very flat segments are EDGE (strict: both area AND mean must be small)
    if abs(area) < edge_th and abs(mean_y) < 5:
        return "EDGE"

    # Check for consistency: area and mean_y should agree on direction
    area_says_in = area > area_th
    area_says_out = area < -area_th
    mean_says_in = mean_y > 10
    mean_says_out = mean_y < -10

    # Only classify when both metrics agree
    if area_says_in and mean_says_in:
        return "IN"
    elif area_says_out and mean_says_out:
        return "OUT"
    elif abs(area) < edge_th and abs(mean_y) < 10:
        # Only EDGE if truly flat
        return "EDGE"
    else:
        # Ambiguous or contradictory evidence → safe WEIRD fallback
        return "WEIRD"


# Test with different thresholds
safe_methods = {
    "Safe (500/100)": lambda s: classify_safe_area(s, area_th=500, edge_th=100),
    "Safe (300/50)": lambda s: classify_safe_area(s, area_th=300, edge_th=50),
    "Safe (800/150)": lambda s: classify_safe_area(s, area_th=800, edge_th=150),
    "Safe (1000/200)": lambda s: classify_safe_area(s, area_th=1000, edge_th=200),
}

rprint("\n[bold]Safe Classification Methods (WEIRD fallback for ambiguous)[/bold]")
rprint(
    f"Expected: IN={expected_in}, OUT={expected_out}, EDGE≤{expected_edge}, WEIRD=0\n"
)

for method_name, classify_fn in safe_methods.items():
    classifications = [classify_fn(s) for s in all_stats_v1]
    counts = Counter(classifications)

    edge_count = counts.get("EDGE", 0)
    excess_edge = max(0, edge_count - expected_edge)

    # Check misclassification
    misclass_results = [check_misclassification(s, classify_fn) for s in all_stats_v1]
    misclass_counts = Counter(misclass_results)
    wrong_polarity = misclass_counts.get("WRONG_POLARITY", 0)
    false_edge = misclass_counts.get("FALSE_EDGE", 0)
    total_errors = wrong_polarity + false_edge

    status = "🟢" if total_errors == 0 else "🔴"

    rprint(
        f"  {method_name:18s}: IN={counts.get('IN', 0):>3} OUT={counts.get('OUT', 0):>3} "
        f"EDGE={edge_count:>3} (excess={excess_edge:>2}) WEIRD={counts.get('WEIRD', 0):>3} | "
        f"errors={total_errors} {status}"
    )

## Summary & Recommendation


In [ ]:
# Final comparison: Current vs Best Safe method
rprint("[bold]Final Comparison[/bold]\n")

current_fn = lambda s: s["shape"].name
best_safe_fn = lambda s: classify_safe_area(s, area_th=500, edge_th=100)

for label, fn in [
    ("Current Algorithm", current_fn),
    ("Proposed Safe Area", best_safe_fn),
]:
    classifications = [fn(s) for s in all_stats_v1]
    counts = Counter(classifications)

    edge_count = counts.get("EDGE", 0)
    excess_edge = max(0, edge_count - expected_edge)

    misclass_results = [check_misclassification(s, fn) for s in all_stats_v1]
    misclass_counts = Counter(misclass_results)

    wrong_polarity = misclass_counts.get("WRONG_POLARITY", 0)
    false_edge = misclass_counts.get("FALSE_EDGE", 0)
    total_errors = wrong_polarity + false_edge

    rprint(f"[bold]{label}[/bold]")
    rprint(
        f"  Distribution: IN={counts.get('IN', 0)} OUT={counts.get('OUT', 0)} "
        f"EDGE={edge_count} (max expected={expected_edge}, excess={excess_edge}) "
        f"WEIRD={counts.get('WEIRD', 0)}"
    )
    rprint(
        f"  Errors: wrong_polarity={wrong_polarity}, false_edge={false_edge}, TOTAL={total_errors}"
    )
    rprint(
        f"  Safe (correct + weird): {misclass_counts.get('correct', 0) + misclass_counts.get('weird_safe', 0)}"
    )
    rprint()

## Deep Dive: Adaptive Method Analysis

The Adaptive method shows promising results:

- IN=78 (expected 82), OUT=71 (expected 82), EDGE=27 (expected 28)
- Only 16 WEIRD (~8%) - much better than current 110

Let's investigate:

1. What are the 16 WEIRD segments? Are they truly ambiguous?
2. Are there any misclassifications hiding?
3. How do the adaptive thresholds behave?


In [ ]:
# Detailed analysis of Adaptive method
def classify_adaptive_detailed(stats):
    """Adaptive classification with detailed breakdown."""
    y_vals = stats["y_vals"]
    flat_th = max(10, np.std(y_vals) * 1.5)
    count_th = max(3, len(y_vals) * 0.05)

    out_count = (y_vals < -flat_th).sum()
    in_count = (y_vals > flat_th).sum()

    is_out = bool(out_count > count_th)
    is_in = bool(in_count > count_th)

    match (is_out, is_in):
        case (True, False):
            result = "OUT"
        case (False, True):
            result = "IN"
        case (False, False):
            result = "EDGE"
        case (True, True):
            result = "WEIRD"

    return {
        "classification": result,
        "flat_th": flat_th,
        "count_th": count_th,
        "out_count": out_count,
        "in_count": in_count,
        "is_out": is_out,
        "is_in": is_in,
        "std_y": np.std(y_vals),
        "n_points": len(y_vals),
    }


# Analyze all segments with adaptive method
adaptive_results = []
for stats in all_stats_v1:
    result = classify_adaptive_detailed(stats)
    result["mean_y"] = stats["mean_y"]
    result["signed_area"] = stats["signed_area"]
    result["piece_id"] = stats["piece_id"]
    result["seg_id"] = stats["seg_id"]
    result["original_shape"] = stats["shape"].name
    adaptive_results.append(result)

# Summary
classifications = Counter(r["classification"] for r in adaptive_results)
rprint("[bold]Adaptive Method Summary[/bold]")
rprint(f"  IN:    {classifications['IN']:>3} (expected {expected_in})")
rprint(f"  OUT:   {classifications['OUT']:>3} (expected {expected_out})")
rprint(f"  EDGE:  {classifications['EDGE']:>3} (expected {expected_edge})")
rprint(f"  WEIRD: {classifications['WEIRD']:>3} (expected 0)")

In [ ]:
# Investigate the 16 WEIRD segments from Adaptive method
weird_adaptive = [r for r in adaptive_results if r["classification"] == "WEIRD"]

rprint(f"\n[bold]The {len(weird_adaptive)} WEIRD segments from Adaptive[/bold]")
rprint(
    "These have points on BOTH sides of the center line (is_out AND is_in both True)\n"
)

for i, w in enumerate(weird_adaptive):
    rprint(f"[{i + 1:>2}] Piece {w['piece_id']}, Seg {w['seg_id']}")
    rprint(f"     mean_y={w['mean_y']:>7.1f}, area={w['signed_area']:>8.1f}")
    rprint(f"     flat_th={w['flat_th']:>5.1f}, count_th={w['count_th']:>4.1f}")
    rprint(f"     out_count={w['out_count']:>3}, in_count={w['in_count']:>3}")
    rprint(f"     Original: {w['original_shape']}")
    rprint()

In [ ]:
# Check for errors in Adaptive: compare mean_y direction with classification
rprint("[bold]Adaptive Error Analysis[/bold]")
rprint("Checking if adaptive classifications match the dominant direction (mean_y)\n")

errors = {
    "wrong_polarity": [],
    "false_edge": [],
    "correct": [],
    "weird_safe": [],
    "ambiguous": [],
}

for r in adaptive_results:
    mean_y = r["mean_y"]
    cls = r["classification"]

    # Determine expected based on mean_y
    if abs(mean_y) < 5:
        errors["ambiguous"].append(r)
        continue

    expected = "IN" if mean_y > 0 else "OUT"

    if cls == "WEIRD":
        errors["weird_safe"].append(r)
    elif cls == "EDGE":
        if abs(mean_y) > 10:
            errors["false_edge"].append(r)
        else:
            errors["ambiguous"].append(r)
    elif cls == expected:
        errors["correct"].append(r)
    else:
        errors["wrong_polarity"].append(r)

rprint(f"  Correct IN/OUT:    {len(errors['correct']):>3}")
rprint(f"  WEIRD (safe):      {len(errors['weird_safe']):>3}")
rprint(f"  Ambiguous:         {len(errors['ambiguous']):>3}")
rprint(f"  [red]Wrong polarity:  {len(errors['wrong_polarity']):>3}[/red]")
rprint(f"  [red]False EDGE:      {len(errors['false_edge']):>3}[/red]")

total_errors = len(errors["wrong_polarity"]) + len(errors["false_edge"])
rprint(f"\n  [bold]TOTAL ERRORS: {total_errors}[/bold]")

In [ ]:
# If there are wrong polarity errors, show them
if errors["wrong_polarity"]:
    rprint(
        "\n[bold red]WRONG POLARITY ERRORS (Adaptive classified opposite direction)[/bold red]"
    )
    for r in errors["wrong_polarity"]:
        expected = "IN" if r["mean_y"] > 0 else "OUT"
        rprint(
            f"  Piece {r['piece_id']}, Seg {r['seg_id']}: classified {r['classification']}, expected {expected}"
        )
        rprint(f"    mean_y={r['mean_y']:.1f}, area={r['signed_area']:.1f}")
        rprint(f"    out_count={r['out_count']}, in_count={r['in_count']}")
else:
    rprint("\n[green]✓ No wrong polarity errors![/green]")

if errors["false_edge"]:
    rprint(
        "\n[bold red]FALSE EDGE ERRORS (Adaptive classified as EDGE but clearly IN/OUT)[/bold red]"
    )
    for r in errors["false_edge"]:
        expected = "IN" if r["mean_y"] > 0 else "OUT"
        rprint(
            f"  Piece {r['piece_id']}, Seg {r['seg_id']}: classified EDGE, expected {expected}"
        )
        rprint(f"    mean_y={r['mean_y']:.1f}, area={r['signed_area']:.1f}")
else:
    rprint("\n[green]✓ No false EDGE errors![/green]")

In [ ]:
# Visualize the WEIRD segments from Adaptive method
# These are the "truly ambiguous" ones that have points on both sides

if weird_adaptive:
    n_to_show = min(6, len(weird_adaptive))
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()

    # Get the actual segment objects for these WEIRD ones
    weird_segs_adaptive = []
    for w in weird_adaptive[:n_to_show]:
        for piece in manager_v1.get_pieces_ls():
            if piece.piece_id == w["piece_id"]:
                for seg_id, seg in piece.segments.items():
                    if seg_id == w["seg_id"]:
                        weird_segs_adaptive.append((w, seg))
                        break

    for i, (w, seg) in enumerate(weird_segs_adaptive[:6]):
        ax = axes[i]
        pts = seg.points.squeeze()

        ax.plot(pts[:, 0], pts[:, 1], "b-", linewidth=2)
        ax.scatter(pts[0, 0], pts[0, 1], c="green", s=100, zorder=5, label="Start")
        ax.scatter(pts[-1, 0], pts[-1, 1], c="red", s=100, zorder=5, label="End")

        ax.set_title(
            f"Piece {w['piece_id']},\nSeg {w['seg_id']}\n"
            f"mean_y={w['mean_y']:.1f}, area={w['signed_area']:.0f}\n"
            f"out={w['out_count']}, in={w['in_count']} (th={w['count_th']:.1f})"
        )
        ax.set_aspect("equal")
        ax.grid(True, alpha=0.3)

    # Hide unused axes
    for j in range(len(weird_segs_adaptive), 6):
        axes[j].set_visible(False)

    plt.suptitle("Segments classified as WEIRD by Adaptive method", fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
# Distribution of adaptive thresholds
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# flat_th distribution
flat_ths = [r["flat_th"] for r in adaptive_results]
axes[0].hist(flat_ths, bins=30, edgecolor="black", alpha=0.7)
axes[0].axvline(x=20, color="red", linestyle="--", label="Current fixed (20)")
axes[0].set_xlabel("flat_th (adaptive)")
axes[0].set_ylabel("Count")
axes[0].set_title(
    f"flat_th distribution\nmin={min(flat_ths):.1f}, max={max(flat_ths):.1f}, mean={np.mean(flat_ths):.1f}"
)
axes[0].legend()

# count_th distribution
count_ths = [r["count_th"] for r in adaptive_results]
axes[1].hist(count_ths, bins=30, edgecolor="black", alpha=0.7)
axes[1].axvline(x=5, color="red", linestyle="--", label="Current fixed (5)")
axes[1].set_xlabel("count_th (adaptive)")
axes[1].set_ylabel("Count")
axes[1].set_title(
    f"count_th distribution\nmin={min(count_ths):.1f}, max={max(count_ths):.1f}, mean={np.mean(count_ths):.1f}"
)
axes[1].legend()

# std_y distribution (drives flat_th)
std_ys = [r["std_y"] for r in adaptive_results]
axes[2].hist(std_ys, bins=30, edgecolor="black", alpha=0.7)
axes[2].set_xlabel("std(y_vals)")
axes[2].set_ylabel("Count")
axes[2].set_title(
    f"Standard deviation of y values\nmin={min(std_ys):.1f}, max={max(std_ys):.1f}"
)

plt.tight_layout()
plt.show()

rprint(f"\n[bold]Adaptive threshold statistics:[/bold]")
rprint(
    f"  flat_th:  min={min(flat_ths):.1f}, max={max(flat_ths):.1f}, mean={np.mean(flat_ths):.1f} (fixed=20)"
)
rprint(
    f"  count_th: min={min(count_ths):.1f}, max={max(count_ths):.1f}, mean={np.mean(count_ths):.1f} (fixed=5)"
)

In [ ]:
# Compare Current vs Adaptive on the same scatter plot
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, (method_name, method_fn) in zip(
    axes,
    [
        ("Current Algorithm", lambda s: s["shape"].name),
        ("Adaptive Method", classify_adaptive),
    ],
):
    for stats in all_stats_v1:
        cls = method_fn(stats)
        color = class_colors.get(cls, "gray")
        ax.scatter(stats["mean_y"], stats["signed_area"], c=color, alpha=0.5, s=40)

    ax.axhline(y=0, color="gray", linestyle="--", alpha=0.3)
    ax.axvline(x=0, color="gray", linestyle="--", alpha=0.3)

    classifications = [method_fn(s) for s in all_stats_v1]
    counts = Counter(classifications)

    ax.set_title(
        f"{method_name}\nIN={counts.get('IN', 0)} OUT={counts.get('OUT', 0)} "
        f"EDGE={counts.get('EDGE', 0)} WEIRD={counts.get('WEIRD', 0)}"
    )
    ax.set_xlabel("Mean Y (positive=IN, negative=OUT)")
    ax.set_ylabel("Signed Area")
    ax.grid(True, alpha=0.3)

# Legend
from matplotlib.patches import Patch

legend_elements = [Patch(facecolor=c, label=l) for l, c in class_colors.items()]
fig.legend(handles=legend_elements, loc="upper right", bbox_to_anchor=(0.99, 0.99))

plt.suptitle("Current vs Adaptive Classification", fontsize=14)
plt.tight_layout()
plt.show()

## Adaptive Method Verdict

Based on the analysis above:

- **Errors**: How many wrong polarity / false EDGE?
- **WEIRD segments**: Are they genuinely ambiguous (points on both sides)?
- **Threshold behavior**: Do adaptive thresholds make sense?

If Adaptive has 0 errors and only classifies genuinely ambiguous segments as WEIRD, it may be the best choice.


In [ ]:
# Final verdict on Adaptive method
rprint("[bold]ADAPTIVE METHOD FINAL VERDICT[/bold]\n")

# Count errors
adaptive_wrong_pol = len(errors["wrong_polarity"])
adaptive_false_edge = len(errors["false_edge"])
adaptive_total_errors = adaptive_wrong_pol + adaptive_false_edge

# Get classifications
adaptive_cls = Counter(r["classification"] for r in adaptive_results)

rprint("[bold]Classification counts:[/bold]")
rprint(
    f"  IN:    {adaptive_cls['IN']:>3} / {expected_in} expected (diff: {adaptive_cls['IN'] - expected_in:+d})"
)
rprint(
    f"  OUT:   {adaptive_cls['OUT']:>3} / {expected_out} expected (diff: {adaptive_cls['OUT'] - expected_out:+d})"
)
rprint(
    f"  EDGE:  {adaptive_cls['EDGE']:>3} / {expected_edge} expected (diff: {adaptive_cls['EDGE'] - expected_edge:+d})"
)
rprint(f"  WEIRD: {adaptive_cls['WEIRD']:>3} / 0 expected")

rprint(f"\n[bold]Error analysis:[/bold]")
rprint(f"  Wrong polarity (IN↔OUT): {adaptive_wrong_pol}")
rprint(f"  False EDGE (IN/OUT→EDGE): {adaptive_false_edge}")
rprint(f"  [bold]TOTAL ERRORS: {adaptive_total_errors}[/bold]")

if adaptive_total_errors == 0:
    rprint("\n[bold green]✓ ADAPTIVE IS SAFE - No misclassifications![/bold green]")
    rprint(
        f"  The {adaptive_cls['WEIRD']} WEIRD classifications are genuinely ambiguous segments."
    )
    rprint("  This is the BEST outcome: minimize errors, accept some safe WEIRD.")
else:
    rprint(f"\n[bold red]⚠ ADAPTIVE HAS {adaptive_total_errors} ERRORS[/bold red]")

# Compare to current
current_cls = Counter(s["shape"].name for s in all_stats_v1)
rprint(f"\n[bold]Comparison to Current algorithm:[/bold]")
rprint(f"  Current WEIRD:  {current_cls['WEIRD']:>3} (57%)")
rprint(
    f"  Adaptive WEIRD: {adaptive_cls['WEIRD']:>3} ({100 * adaptive_cls['WEIRD'] / len(all_stats_v1):.0f}%)"
)
rprint(
    f"  Reduction: {current_cls['WEIRD'] - adaptive_cls['WEIRD']} fewer WEIRD classifications"
)